In [1]:
import sys
if ".." not in sys.path:
    sys.path.append("..")
import openpyxl
from energydata import energinet

import importlib
energinet = importlib.reload(energinet)

In [2]:
descriptor = {"resource_id": "connectionpointsingrid"}
service = energinet.EnergiDataServiceDk(descriptor)

In [3]:
list(energinet.fetch_edh(descriptor, 1))

[]

In [4]:
list(service.fetch_data())

[{'_id': 1001,
  'ConnectionPointCode': 'ØLS',
  'ConnectionPointName': 'Ølstykkegård',
  'MunicipalityNo': '240',
  'Municipality': 'Egedal',
  'RegionName': 'Hovedstaden',
  'ConnectionPointLatitude': 55.77148,
  'ConnectionPointLongitude': 12.143712,
  'NetCompanyName': 'REGIONALE NET',
  'region_code': 'dk_mun_240'},
 {'_id': 20,
  'ConnectionPointCode': 'AMK',
  'ConnectionPointName': 'Amagerkbst',
  'MunicipalityNo': '101',
  'Municipality': 'København',
  'RegionName': 'Hovedstaden',
  'ConnectionPointLatitude': 55.641284,
  'ConnectionPointLongitude': 12.609259,
  'NetCompanyName': 'KE TRANSMISSION',
  'region_code': 'dk_mun_101'},
 {'_id': 22,
  'ConnectionPointCode': 'AMV',
  'ConnectionPointName': 'Amagerværket',
  'MunicipalityNo': '101',
  'Municipality': 'København',
  'RegionName': 'Hovedstaden',
  'ConnectionPointLatitude': 55.685241,
  'ConnectionPointLongitude': 12.624494,
  'NetCompanyName': 'KE TRANSMISSION',
  'region_code': 'dk_mun_101'},
 {'_id': 23,
  'Connectio

In [5]:
metadata = service.fetch_metadata()
data = service.fetch_data()

In [6]:
connection_points = {
    row["ConnectionPointCode"] : row for row in data
}

In [7]:
{k: v for k, v in connection_points.items() if "strup" in v["ConnectionPointName"].lower()}

{'ESK': {'_id': 171,
  'ConnectionPointCode': 'ESK',
  'ConnectionPointName': 'Eskilstrup',
  'MunicipalityNo': '376',
  'Municipality': 'Guldborgsund',
  'RegionName': 'Sjælland',
  'ConnectionPointLatitude': 54.849705,
  'ConnectionPointLongitude': 11.893968,
  'NetCompanyName': 'SEAS/NVE TRANSMISSION',
  'region_code': 'dk_mun_376'},
 'EST': {'_id': 173,
  'ConnectionPointCode': 'EST',
  'ConnectionPointName': 'Estrupvej',
  'MunicipalityNo': '561',
  'Municipality': 'Esbjerg',
  'RegionName': 'Syddanmark',
  'ConnectionPointLatitude': 55.462691,
  'ConnectionPointLongitude': 8.474662,
  'NetCompanyName': 'Vestjyske Net A/S',
  'region_code': 'dk_mun_561'},
 'FARV': {'_id': 180,
  'ConnectionPointCode': 'FARV',
  'ConnectionPointName': 'Farstrup',
  'MunicipalityNo': '480',
  'Municipality': 'Nordfyn',
  'RegionName': 'Syddanmark',
  'ConnectionPointLatitude': 55.46349,
  'ConnectionPointLongitude': 10.1906,
  'NetCompanyName': 'VORES ELNET A/S',
  'region_code': 'dk_mun_480'},
 'HU

In [8]:
# ws = openpyxl.load_workbook("../Energinet Extra Codes.xlsx").active

# [column_names, *rows] = list(ws.values)
# extra_station_codes = {}
# for row in rows:
#     d = dict(zip(column_names, row))
#     if d["Code"] is not None:
#         extra_station_codes[d["Name"]] = d["Code"]

In [9]:
# extra_station_codes

In [10]:
extra_station_codes = {
    'A2': 'BJS',
    'AMF_132_F': 'AMV',
    'HASØ132_F': 'HAS',
    'ISHN132_N-$': 'ISH',
    'MRP1': 'TEG',
    'VSAK132_S1': 'VAL',
    'BIX_150_S1': 'BIL',
    'FRX_150_S1': 'FRT',
    'I4': 'ABS',
    'KLT_150_S1': 'KLF',
    'VKHI012_SFIK': 'VKE',
    'term_KT51-HT3': 'TJE'
}

In [13]:
import io
import urllib.request

In [14]:
netdata_url = "https://energinet.dk/-/media/76C3D434A7E2499F94E7DB66D3A2DE21.xlsx?la=da&hash=638E165AA57F627409CB9394D56D53614F5D7336"

with urllib.request.urlopen(netdata_url) as fileobj:
    buf = io.BytesIO()
    buf.write(fileobj.read())

buf.seek(0)
wb = openpyxl.load_workbook(buf)

In [15]:
wb.sheetnames

['Bus',
 'Line',
 'Transformer2',
 'Transformer3',
 'Generator',
 'Load',
 'Shunt',
 'HVDC',
 'PFModelInfo',
 'Ark1',
 'Ark2',
 'Ark3']

In [16]:
import networkx as nx

In [17]:
def get_sheet_data(ws):
    [_1, _2, _3, header, *rows] = ws.values
    for row in rows:
        yield {k: v for k, v in zip(header, row)}

In [18]:
import re
def match_bus_to_code(d, codes, extra_codes):
    extensions = {
        "ENDKW": "V",
        "ENDKE": "Ø"
    }
    name = d["Bus Name"]
    if name in extra_codes and extra_codes[name] in codes:
        return extra_codes[name]
    name_regex = re.compile(r"([^0-9_]+)_?(\d*)_?(.*)")
    match = name_regex.match(name)
    if not match:
        return
    bus_code = match.groups()[0]
    if bus_code in codes:
        return bus_code
    if d["Location Name"] in extensions:
        area_code = bus_code + extensions[d["Location Name"]]
        if area_code in codes:
            return area_code

In [19]:
energy_grid = nx.Graph()

In [20]:
bus_labels = {}
for d in get_sheet_data(wb["Bus"]):
    node_label = ("Bus", d["Bus Name"])
    bus_labels[d["Bus Index"]] = node_label
    attributes = {k: v for k, v in d.items()}
    connection_code = match_bus_to_code(d, list(connection_points), extra_station_codes)
    if connection_code is not None:
        attributes.update(connection_points[connection_code])
    energy_grid.add_nodes_from([(node_label, attributes)])

In [21]:
for d in get_sheet_data(wb["Line"]):
    energy_grid.add_edges_from([(bus_labels[d["Node 1"]], bus_labels[d["Node 2"]], d)])

In [22]:
for d in get_sheet_data(wb["Transformer2"]):
    node_label = ("Transformer3", d["2-Transformer Name"])    
    energy_grid.add_nodes_from([(node_label, d)])
    energy_grid.add_edges_from([
        (bus_labels[d["High.V Bus Index"]], node_label), 
        (bus_labels[d["Low.V Bus Index"]], node_label)
    ])

In [23]:
for d in get_sheet_data(wb["Transformer3"]):
    node_label = ("Transformer3", d["3_Transformer Name"])
    energy_grid.add_nodes_from([(node_label, d)])
    edges = [
        (bus_labels[d["High.V Bus Index"]], node_label),
        (bus_labels[d["Mid.V Bus Index"]], node_label),        
        (bus_labels[d["Low.V Bus Index"]], node_label)
    ]
    energy_grid.add_edges_from(edges)

In [24]:
for d in get_sheet_data(wb["Generator"]):
    node_label = ("Generator", d["Generator Name"])
    energy_grid.add_nodes_from([(node_label, d)])
    energy_grid.add_edge(node_label, bus_labels[d["Bus Index"]])

In [25]:
for d in get_sheet_data(wb["Load"]):
    node_label = ("Load", d["Load Name"])
    energy_grid.add_nodes_from([(node_label, d)])
    energy_grid.add_edge(node_label, bus_labels[d["Load Index"]])

In [26]:
for d in get_sheet_data(wb["Shunt"]):
    node_label = ("Shunt", d["Shunt Name"])
    energy_grid.add_nodes_from([(node_label, d)])
    energy_grid.add_edge(node_label, bus_labels[d["Bus Index"]])

In [27]:
for d in get_sheet_data(wb["HVDC"]):
    node_label = ("HVDC", d["HVDC Name"])
    energy_grid.add_nodes_from([(node_label, d)])
    energy_grid.add_edge(node_label, bus_labels[d["Bus Index"]])

In [28]:
[len(x) for x in nx.connected_components(energy_grid)]

[451, 561, 1, 1]

In [29]:
[x for x in nx.connected_components(energy_grid) if len(x) == 1]

[{('Bus', 'EDR_400_NIE1')}, {('Bus', 'EDR_400_NIE1(1)')}]

In [89]:
municipality_nodes = {k: v["region_code"] for k, v in energy_grid.nodes.items() if "region_code" in v}
municipality_nodes

{('Bus', 'A2'): 'dk_mun_259',
 ('Bus', 'ABS_150_S2-'): 'dk_mun_420',
 ('Bus', 'ADL_150_S1'): 'dk_mun_851',
 ('Bus', 'ALL_132_F1'): 'dk_mun_201',
 ('Bus', 'AMF_132_F'): 'dk_mun_101',
 ('Bus', 'AMK_132_A'): 'dk_mun_101',
 ('Bus', 'AMV_010_SFIK'): 'dk_mun_101',
 ('Bus', 'AMV_0173_SFIK'): 'dk_mun_101',
 ('Bus', 'AMV_132_A'): 'dk_mun_101',
 ('Bus', 'AND_150_S2'): 'dk_mun_575',
 ('Bus', 'ASR_150_S2'): 'dk_mun_756',
 ('Bus', 'ASR_400-ZL-'): 'dk_mun_756',
 ('Bus', 'ASR_400_S1'): 'dk_mun_756',
 ('Bus', 'ASV_0122_SFIK'): 'dk_mun_326',
 ('Bus', 'ASV_0215_SFIK'): 'dk_mun_326',
 ('Bus', 'ASV_132_F'): 'dk_mun_326',
 ('Bus', 'ASV_4001_SFIK'): 'dk_mun_326',
 ('Bus', 'ASV_400_K'): 'dk_mun_326',
 ('Bus', 'AVV_0122_SFIK'): 'dk_mun_167',
 ('Bus', 'AVV_0123_SFIK'): 'dk_mun_167',
 ('Bus', 'AVV_0171_SFIK'): 'dk_mun_167',
 ('Bus', 'AVV_0201_SFIK'): 'dk_mun_167',
 ('Bus', 'AVV_132_F'): 'dk_mun_167',
 ('Bus', 'AVV_132_SFIK'): 'dk_mun_167',
 ('Bus', 'AVV_400_SK.K'): 'dk_mun_167',
 ('Bus', 'BAG_132_F'): 'dk_mun_1

In [90]:
municipality_connections = set()
for n1, n2 in energy_grid.edges:
    try:
        m1 = municipality_nodes[n1]
        m2 = municipality_nodes[n2]
        if m2 < m1:
            m1, m2 = m2, m1
        municipality_connections.add((m1, m2))
    except KeyError:
        pass
municipality_connections

{('dk_mun_101', 'dk_mun_101'),
 ('dk_mun_101', 'dk_mun_159'),
 ('dk_mun_101', 'dk_mun_167'),
 ('dk_mun_101', 'dk_mun_185'),
 ('dk_mun_151', 'dk_mun_240'),
 ('dk_mun_159', 'dk_mun_159'),
 ('dk_mun_159', 'dk_mun_161'),
 ('dk_mun_159', 'dk_mun_240'),
 ('dk_mun_161', 'dk_mun_161'),
 ('dk_mun_167', 'dk_mun_183'),
 ('dk_mun_169', 'dk_mun_183'),
 ('dk_mun_169', 'dk_mun_240'),
 ('dk_mun_169', 'dk_mun_265'),
 ('dk_mun_169', 'dk_mun_350'),
 ('dk_mun_183', 'dk_mun_240'),
 ('dk_mun_183', 'dk_mun_259'),
 ('dk_mun_183', 'dk_mun_265'),
 ('dk_mun_201', 'dk_mun_217'),
 ('dk_mun_201', 'dk_mun_219'),
 ('dk_mun_217', 'dk_mun_219'),
 ('dk_mun_217', 'dk_mun_260'),
 ('dk_mun_219', 'dk_mun_219'),
 ('dk_mun_219', 'dk_mun_240'),
 ('dk_mun_219', 'dk_mun_260'),
 ('dk_mun_240', 'dk_mun_240'),
 ('dk_mun_240', 'dk_mun_250'),
 ('dk_mun_240', 'dk_mun_259'),
 ('dk_mun_250', 'dk_mun_250'),
 ('dk_mun_250', 'dk_mun_350'),
 ('dk_mun_253', 'dk_mun_269'),
 ('dk_mun_260', 'dk_mun_260'),
 ('dk_mun_265', 'dk_mun_265'),
 ('dk_mu

In [91]:
energy_grid.nodes[list(municipality_nodes)[0]]

{'Bus Index': 76,
 'Bus Name': 'A2',
 'Station Full Name': 'Bjæverskov',
 'Location Name': 'ENDKE',
 'Voltage base[kV]': '232,0',
 'Voltage min[pu]': '0,00',
 'Voltage max[pu]': '1,05',
 '_id': 76,
 'ConnectionPointCode': 'BJS',
 'ConnectionPointName': 'Bjæverskov',
 'MunicipalityNo': '259',
 'Municipality': 'Køge',
 'RegionName': 'Sjælland',
 'ConnectionPointLatitude': 55.45053,
 'ConnectionPointLongitude': 12.00791,
 'NetCompanyName': 'SEAS/NVE TRANSMISSION',
 'region_code': 'dk_mun_259'}

In [92]:
import sqltables
db = sqltables.Database()
latlon = db.create_table(column_names=["region_code", "lat", "lon"])
latlon.insert([[x["region_code"], x["ConnectionPointLatitude"], x["ConnectionPointLongitude"]] 
               for x in (energy_grid.nodes[k] for k in municipality_nodes.keys())])

In [93]:
mun_latlon = latlon.table("""select region_code, avg(lat) as lat, avg(lon) as lon from _ group by region_code""")
mun_points = {row.region_code: (row.lat, row.lon) for row in mun_latlon}

In [94]:
mun_points

{'dk_mun_101': (55.67414544444444, 12.590287444444444),
 'dk_mun_151': (55.744171, 12.32608),
 'dk_mun_159': (55.7501205, 12.485704),
 'dk_mun_161': (55.686573, 12.397478),
 'dk_mun_167': (55.604988999999996, 12.480812),
 'dk_mun_169': (55.660772, 12.319345),
 'dk_mun_183': (55.6346, 12.32171),
 'dk_mun_185': (55.631542, 12.657965),
 'dk_mun_201': (55.885227, 12.338456),
 'dk_mun_217': (56.028681, 12.563658),
 'dk_mun_219': (55.8955735, 12.196541750000002),
 'dk_mun_240': (55.755691, 12.198973),
 'dk_mun_250': (55.80958483333333, 11.896245666666665),
 'dk_mun_253': (55.585099, 12.258356),
 'dk_mun_259': (55.45053, 12.00791),
 'dk_mun_260': (55.961136, 12.019283),
 'dk_mun_265': (55.6285435, 12.1149735),
 'dk_mun_269': (55.54376, 12.20485),
 'dk_mun_306': (55.871017, 11.603077),
 'dk_mun_316': (55.717711, 11.491859),
 'dk_mun_326': (55.66188011111111, 11.087549777777777),
 'dk_mun_340': (55.545725, 11.641376),
 'dk_mun_350': (55.645783333333334, 11.932815666666668),
 'dk_mun_360': (54.8

In [95]:
connection_points = {(m1, m2): (mun_points[m1], mun_points[m2]) for m1, m2 in municipality_connections}

In [96]:
import json
with open("connection_points.json", "w+") as fileobj:
    json.dump(list(connection_points.values()), fileobj)